# Enunciado

In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np

## Preparación de datos (1.0pt)

Explore el archivo `covid19_confirmados.csv` y utilice la función `pd.read_csv` para importarlo como un `DataFrame` considerando las siguientes indicaciones:

1. Utilice el nombre de país como índice.
1. Elimine las columnas `Lat` y `Long`.
1. Use TimeStamps para las columnas asociadas a fechas.
1. Reduzca la columna `Province/State`. El DataFrame resultante debe tener los totales a nivel país.

In [ ]:
def crea_dataframe(data): #data -> ruta del archivo
    #creacion dataframe, elimina columnas Lat y Long 
    df = pd.read_csv(data,index_col="Country/Region",parse_dates=True).drop(["Lat","Long"],axis=1)

    #reduce la columna province/state con un groupby manteniendo los totales por pais
    df = df.groupby(level=0).sum(numeric_only=True)

    #parsear fechas
    fechas = []
    for i in df.columns:
        fechas.append(pd.to_datetime(i)) #añade a una lista la transformacion a timestamp de cada fecha
    df.columns = fechas #asigna a columnas la lista de fechas parseadas a timestamp

    return df
        

In [ ]:
df_confirmados = crea_dataframe("data/covid19_confirmados.csv") #creacion del df
df_confirmados

Explore el archivo `population_by_country_2020.csv` y utilice la función `pd.read_csv` de pandas para importarlo como un `DataFrame`. Utilice el nombre de país como índice principal. 

Nota: Algunos siglas o nombres podrían no calzar con el `DataFrame` anterior, explore y corrija programaticamente dichos errores.

In [ ]:
df_population = pd.read_csv("data/poblacion_mundial2020.csv") #dataframe de population
df_population.rename(columns={"Country (or dependency)":"Country/Region"}, inplace=True) #Cambio del nombre de el indice de la columna
df_population.set_index("Country/Region",inplace=True) #asignacion de "nuevo" nombre

Realice un `merge` de los `DataFrame` anteriores. El objetivo es asignar un valor de `Population` a cada elemento del primer `DataFrame`. Indique las filas donde no se puede realizar el `merge` y luego descártelas de su `DataFrame` final.

In [ ]:
def eliminar_error(df,df_population):
    valores_error = []
    for pais in df.index:
        if pais not in df_population.index:
            valores_error.append(pais)
            df.drop(labels=pais, axis=0, inplace=True)
    # display("Los siguientes datos NO se encuentran en population: ",valores_error) #descomentar esta linea para verificar los paises con errores

In [ ]:
error = ['Antarctica', #no existen datos en poblacion_mundial
 'Burma', #el nombre es Myanmar en poblacion_mundial
 'Congo (Brazzaville)', #pais de africa central: Congo -> conocido como Congo brazzaville
 'Congo (Kinshasa)', #nombre R.D Congo
 "Cote d'Ivoire", #error semantico Côte d'Ivoire, cambiar a Cote d'Ivoire
 'Czechia', #diferente nombre (Czech Republic (Czechia)) cambiar a Czechia
 'Diamond Princess', #no es un pais?
 'Korea, South', #correccion a South Korea
 'Kosovo', #no existen datos en poblacion_mundial
 'MS Zaandam', #no es un pais?
 'Saint Kitts and Nevis', #error semantico en poblacion mundial el and estaba representado por un &
 'Saint Vincent and the Grenadines', #cambio de nombre St. Vincent & Grenadines -> Saint Vincent and the Grenadines
 'Sao Tome and Principe', #Sao Tome & Principe cambiar el & por and
 'Summer Olympics 2020', #no es pais
 'Taiwan*', #correccion del simbolo *
 'US', #cambio en archivos covid19_  a united states 
 'West Bank and Gaza', #no existen datos en poblacion mundial
 'Winter Olympics 2022'] #no es un pais

In [ ]:
eliminar_error(df_confirmados,df_population)
df_confirmados = pd.merge(df_confirmados,df_population, on="Country/Region")
display(df_confirmados)


Repita el procedimiento para los archivos `covid19_recuperados.csv` y `covid19_muertes.csv`.

In [ ]:
#creacion de siguientes dataframe y merge con population
df_recuperados = crea_dataframe("data/covid19_recuperados.csv") #creacion del df
eliminar_error(df_recuperados,df_population)
df_recuperados = pd.merge(df_recuperados,df_population, on="Country/Region") #merge

df_muertes = crea_dataframe("data/covid19_muertes.csv") #creacion del df
eliminar_error(df_muertes,df_population)
df_muertes = pd.merge(df_muertes,df_population, on="Country/Region") #merge

display(df_recuperados)
display(df_muertes)

## Análisis global (1.5pt)

Escriba una función que reciba una fecha y que retorne las siguientes series: 

1. Tasa de incidencia: Casos confirmados por 100.000 dividido población total.
1. Tasa de recuperación: Casos recuperados por 100.000 dividido población total.
1. Tasa de mortalidad: Casos decesos por 100.000 dividido población total.

Ahora, considerando los siguientes dos puntos temporales:

- 22 de Enero de 2021.
- 22 de Enero de 2022.

Muestre los nombres y los valores de los 3 países con:

1. La mayor y menor tasa de incidencia, respectivamente.
1. La mayor y menor tasa de recuperación, respectivamente.
1. La mayor y menor tasa de mortalidad, respectivamente.

Muestre también la posición de Chile en el ranking.

In [ ]:
def series_fecha(fecha):
    tasa_incidencia = (df_confirmados[fecha]*100000)/df_confirmados["Population (2020)"]
    tasa_recuperacion = (df_recuperados[fecha]*100000)/df_recuperados["Population (2020)"]
    tasa_mortalidad = (df_muertes[fecha]*100000)/df_muertes["Population (2020)"]

    dicc_series = {"tasa incidencias":tasa_incidencia,
                    "tasa recuperacion": tasa_recuperacion,
                    "tasa mortalidad": tasa_mortalidad
                    }
    return dicc_series


In [ ]:
#analsis de fechas 22 de Enero de 2021 y 22 de Enero de 2022
fecha1 = series_fecha(pd.to_datetime("2021-01-22"))
fecha2 = series_fecha(pd.to_datetime("2022-01-22"))

# display(fecha1["tasa mortalidad"]) #acceder a los datos especificos del diccionario

In [ ]:
def serie_min_max(df,tipo):
    #mayor
    df_max = (pd.DataFrame(df[tipo]).set_axis([tipo],axis=1).sort_values(tipo,ascending=False)).head(3)
    df_max.insert(1,"rank",df_max.rank(ascending=False))
    
    chile1 = (pd.DataFrame(df[tipo]).set_axis([tipo],axis=1).sort_values(tipo,ascending=False))
    chile1.insert(1,"rank",chile1.rank(ascending=False))

    #menor
    df_min = (pd.DataFrame(df[tipo]).set_axis([tipo],axis=1).sort_values(tipo,ascending=True)).head(3)
    df_min.insert(1,"rank",df_min.rank(ascending=True))

    display(df_max,pd.DataFrame(chile1.loc["Chile"]),df_min)
    print()

In [ ]:
# display("La mayor y menor tasa de incidencia, respectivamente.")
# print("fecha 1:")
# serie_min_max(fecha1,"tasa incidencias")
# print("fecha 2: ")
# serie_min_max(fecha2,"tasa incidencias")

# display("La mayor y menor tasa de recuperación, respectivamente.")
# print("fecha 1: ")
# serie_min_max(fecha1,"tasa recuperacion")
# print("fecha 2: ")
# serie_min_max(fecha2,"tasa recuperacion")

# display("La mayor y menor tasa de mortalidad, respectivamente.")
# print("fecha 1: ")
# serie_min_max(fecha1,"tasa mortalidad")
# print("fecha 2: ")
# serie_min_max(fecha2,"tasa mortalidad")


#

Ahora realice las siguientes gráficas de nube de punto

- Tasa de incidencia Enero 2021 versus Tasa de incidencia Enero 2022.
- Tasa de mortalidad Enero 2021 versus Tasa de incidencia Enero 2022.

En cada una:

- Utilice color para demarcar el [continente](https://www.kaggle.com/datasets/statchaitya/country-to-continent) de cada país.
- Realice una anotación con el código del pais cercano a su punto: https://country-code.cl/es/.
- Utilice un tamaño relativamente más grande para Chile.
- Considere las buenas prácticas vistas en clases.

In [ ]:
#Continentes y codigo ISO2 archivo generado con chatgpt
continentes = pd.read_csv("data/pais_cod_continente.csv",index_col="Country/Region")

df_confirmados = pd.merge(df_confirmados,continentes,on="Country/Region")
df_recuperados = pd.merge(df_recuperados,continentes,on="Country/Region")
df_muertes = pd.merge(df_muertes,continentes,on="Country/Region")

# display(continentes)

In [ ]:
#Verificacion de la informacion de los dataframe
# display(df_confirmados)
# display(df_recuperados)
# display(df_muertes)

In [ ]:
#nube de puntos

############################################## LISTO ##############################################
def nube_puntos(df1,df2,t1,t2):
    df_1 = df1.copy()
    df_2 = df2.copy()

    df_1.reset_index(level=0, inplace=True)
    df_1.set_index(["Continent","Country/Region"],inplace=True,drop=True)
    df_2.reset_index(level=0, inplace=True)
    df_2.set_index(["Continent","Country/Region"],inplace=True,drop=True)

    df_c1 = df_1.copy()
    df_c2 = df_2.copy()

    continentes_lista = {"South America":"blue",
                        "North America":"Red",
                        "Africa":"Green",
                        "Europe":"Grey",
                        "Asia":"magenta",
                        "Oceania":"orange"}

    fig, ax = plt.subplots(figsize=(14, 8), 
                       sharex=False, sharey=False, tight_layout=True)

    for i, etiqueta in enumerate(continentes_lista):
        # display(i,continentes_lista[etiqueta],etiqueta)
        
        #Calculo de tasas 
        df_c1.loc[etiqueta,[pd.to_datetime("2021-01-22")]] = (df_1[pd.to_datetime("2021-01-22")]*100000)/df_1["Population (2020)"]
        df_c2.loc[etiqueta, [pd.to_datetime("2022-01-22")]] = (df_2[pd.to_datetime("2022-01-22")]*100000)/df_2["Population (2020)"]
        
        ax.scatter(x=df_c1[pd.to_datetime("2021-01-22")].loc[etiqueta],
                   y= df_c2[pd.to_datetime("2022-01-22")].loc[etiqueta],
                   s=30,marker='o',color=continentes_lista[etiqueta],label=etiqueta)
        
        if(etiqueta == "South America"):
            ax.scatter(x=df_c1[pd.to_datetime("2021-01-22")].loc[("South America","Chile")],
                       y= df_c2[pd.to_datetime("2022-01-22")].loc[("South America","Chile")],
                       s=150,marker='*',color="BLACK",label="Chile")
        ax.set_title(t1+" vs "+t2)
        ax.set_xlabel(t1)
        ax.set_ylabel(t2)

    #Etiqueta a cada elemento
    for i in df_c1.index:
        ax.annotate(df_c1.loc[(i[0],i[1]),"ISO2"], 
                    xy=(df_c1[pd.to_datetime("2021-01-22")].loc[(i[0],i[1])],
                        df_c2[pd.to_datetime("2022-01-22")].loc[(i[0],i[1])]));
        
 
    ax.legend()
        

# nube_puntos(df_confirmados,df_confirmados,"Tasa incidencias 2021","Tasa incidencias 2022")
# nube_puntos(df_muertes,df_confirmados,"Tasa mortalidad 2021","Tasa incidencias 2022")


En base a los rankings y gráficas de nube de punto analice y discuta sobre el caso de Chile considerando las siguientes preguntas guía:

- ¿A qué países del mundo se parece más?
- ¿A qué países de América Latina se parece más?
- ¿A qué países de la [OCDE](https://es.wikipedia.org/wiki/Organizaci%C3%B3n_para_la_Cooperaci%C3%B3n_y_el_Desarrollo_Econ%C3%B3micos) se parece más? 


1-
2-
3-


## Análisis comparativo (1.5pt)

Construya una interfaz gráfica usando `ipywidgets` y `matplotlib` que permita realizar comparaciones entre las series de tiempo de COVID-19 de dos países seleccionados arbitrariamente. A modo de simplificación considere sólo los países de la [OCDE](https://es.wikipedia.org/wiki/Organizaci%C3%B3n_para_la_Cooperaci%C3%B3n_y_el_Desarrollo_Econ%C3%B3micos). 

Requerimientos:

1. La interfaz debe incluir controles para escoger dos países a partir de una lista.
1. La interfaz debe incluir tres [tabs](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Tabs), el primero debe mostrar las series de tiempo de ambos países de los casos confirmados, el segundo la de recuperados y el tercero la de los decesos, respectivamente. Use una leyenda para indicar el nombre de los países.
1. La interfaz debe tener un control que permita escoger entre casos acumulados y casos nuevos. Hint: Para los casos nuevos pueden revisar la función [`diff`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.diff.html).
1. La interfaz debe tener un control que permita escoger entre valores absolutos y valores relativos (tasas por 100.000 habitantes).
1. La interfaz debe tener un control que permita escoger entre valores diarios y valores semanales. Para esto último se recomienda usar `groupby` con una [frecuencia lunes a lunes](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#anchored-offsets).
    



In [ ]:
#dataframe de paises OCDE -> datos obtenidos con chatgpt
df_OCDE_confirmados = pd.read_csv("data/paises_OCDE.csv",index_col="Country/Region")
df_OCDE_confirmados = pd.merge(df_confirmados,df_OCDE_confirmados,on="Country/Region")

df_OCDE_recuperados= pd.read_csv("data/paises_OCDE.csv",index_col="Country/Region")
df_OCDE_recuperados = pd.merge(df_recuperados,df_OCDE_recuperados,on="Country/Region")

df_OCDE_muertes = pd.read_csv("data/paises_OCDE.csv",index_col="Country/Region")
df_OCDE_muertes = pd.merge(df_muertes,df_OCDE_muertes,on="Country/Region")

# display(df_OCDE_confirmados)
# display(df_OCDE_recuperados)
# display(df_OCDE_muertes)

In [ ]:
dataframes_OCDE = {0: df_OCDE_confirmados,
                   1: df_OCDE_recuperados,
                   2: df_OCDE_muertes} #ayuda a acceder al df de forma mas facil
plt.ioff()
####################################### Filtros #############################################

def filtro1(df): #casos nuevos o casos acumulados
    if(selector_casos.value == "Casos nuevos"):
        return df.diff(axis=1).clip(lower=0) # diff() calcula la diferencia entre filas (o columnas) adyacentes axis = 0 -> filas axis = 1 columnas
        #clip() limita los valores de una columna a un rango específico.  
        #En este caso, se puede establecer el rango mínimo en cero para eliminar los valores negativos.
        #los valores negativos son dados debido a falla en los datos si fecha1 x-1 y fecha2 x al hacer la diferencia para obtener nuevos
        #casos fecha2 - fecha1 el resultado da negativo
    elif(selector_casos.value == "Casos acumulados"):
        return df
    else:
        return df

def filtro2(df):
    if(selector_valores.value == "Valores absolutos"):
        df.loc[pais1.value]  = df.loc[pais1.value][:810]*100000/df_OCDE_confirmados.loc[pais1.value]["Population (2020)"]
        df.loc[pais2.value] = df.loc[pais2.value][:810]*100000/df_OCDE_confirmados.loc[pais2.value]["Population (2020)"]

        return df
    elif(selector_valores.value == "Valores relativos"):
        return df
    else:
        return df
    
def filtro3(df):
    if(selector_fecha.value == "Valores semanales"):
        fechas = pd.PeriodIndex(df.columns, freq="W-MON", copy = bool).to_timestamp() #Genera un arreglo de tipo datetime con las fechas del dataframe(columnas)
        #con frecuencia semanal con el dia lunes como dia inicial
        df.columns = fechas
        df.groupby(df.columns , axis=1).sum() #agrupar en una suma los valores de las fechas con el mismo nombre de columna(axis=1 es columnas)
        
        return df
    elif(selector_fecha.value == "Valores diarios"):
        return df
    else:
        return df

def crear_grafico(num): #crea el grafico dentro de cada hijo de la tabla
    if(pais1.value == None or pais2.value == None):
        display("selecciona paises")
    elif(pais1.value == pais2.value):
        display("selecciona paises distintos")
    else:
        with tabla.children[num]:
            if(selector_casos.value != None or selector_valores.value != None or selector_fecha.value != None):
                df_OCDE_c = dataframes_OCDE[num].copy()

                #Aplicacion de filtros
                df_OCDE_c = filtro1(df_OCDE_c.iloc[:,: 810])

                df_OCDE_c = filtro2(df_OCDE_c.iloc[:,: 810])

                df_OCDE_c = filtro3(df_OCDE_c.iloc[:,: 810])
                ####################################################

                fig, ax = plt.subplots(figsize=(14, 8), tight_layout=True, facecolor='#EEF')
                
                line, = ax.plot(df_OCDE_c.loc[pais1.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")],label=pais1.value)
                line2, = ax.plot(df_OCDE_c.loc[pais2.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")], label=pais2.value)
                plt.legend()
                ax.set_title(pais1.value+" vs "+pais2.value)
                plt.show()
            else:
                fig, ax = plt.subplots(figsize=(14, 8), tight_layout=True, facecolor='#EEF')
                
                line, = ax.plot(dataframes_OCDE[num].loc[pais1.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")],label=pais1.value)
                line2, = ax.plot(dataframes_OCDE[num].loc[pais2.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")], label=pais2.value)
                plt.legend()
                ax.set_title(pais1.value+" vs "+pais2.value)
                plt.show()

def boton_presionado(dato): #elimina el grafico actual y crea otro grafico con los nuevos valores
    for i in range(0,3):
        with tabla.children[i]:
            tabla.children[i].clear_output()
            crear_grafico(i)

def r_filtros(data):
    selector_casos.value = None
    selector_valores.value = None
    selector_fecha.value = None

########################################## Creacion de botones y tabla ##########################################

pais1 = widgets.Dropdown(value=None, options=df_OCDE_confirmados.index, description='Pais 1')
pais2 = widgets.Dropdown(value=None, options=df_OCDE_confirmados.index, description='Pais 2')

confirmar = widgets.Button(description='Confirmar')
reiniciar_filtros = widgets.Button(description='Reiniciar filtros')

selector_casos = widgets.RadioButtons(
    options=['Casos acumulados', 'Casos nuevos'],
    value=None,
    disabled=False
)

selector_valores = widgets.RadioButtons(
    options=['Valores absolutos', 'Valores relativos'],
    value=None,
    disabled=False
)

selector_fecha = widgets.RadioButtons(
    options=['Valores diarios', 'Valores semanales'],
    value=None,
    disabled=False
)

datos_tabla = ["Tasa incidencias", "Tasa recuperados","Tasa mortalidad"]
tabla = widgets.Tab(children=[widgets.Output() for i in range(len(datos_tabla))])
for i in range(len(datos_tabla)):
    tabla.set_title(i,datos_tabla[i])

output = widgets.Output()

fila1 = widgets.HBox([pais1,pais2,output])
fila_filtro = widgets.VBox([confirmar,reiniciar_filtros])
fila2 = widgets.HBox([selector_casos,selector_valores,selector_fecha,fila_filtro])

display(widgets.VBox([fila1, fila2,output]),tabla)

#################################### Acciones con cada boton ####################################

confirmar.on_click(boton_presionado) #asigna los filtros seleccionados a cada copia del dataframe
reiniciar_filtros.on_click(r_filtros) #asigna none a los valores de los filtros


<p>En Australia en los casos confirmados la fecha 2022-01-30 tiene 2.607.242 casos y para la fecha 22-01-31 tiene 2.580.386 casos 
por lo tanto al hacer el diff entre la fecha2 - fecha 1 da -26.856 esto implica que la grafica se muestren datos negativos <p>

## Dashboard interactivo (1.0pt)

Implemente un *dashboard web* interactivo a partir de la interfaz desarrollada en el punto anterior utilizando la librería [voila](https://github.com/voila-dashboards/voila). Sirva su interfaz en la nube, para esto puede considerar los servicios gratuitos [binder](https://mybinder.org/) o [heroku](https://www.heroku.com/free). Revise la documentación de voila [con respecto a *deployment* en la nube](https://voila.readthedocs.io/en/stable/deploy.html) y este repositorio con [un ejemplo en base a heroku](https://github.com/voila-dashboards/voila-heroku). 

Escriba el link de su dashboard web a continuación:

https://mybinder.org/v2/gh/FelipePrieto22/dashboard/HEAD?urlpath=voila%2Frender%2Findex.ipynb

## Discusión final (1.0pt)

En base a su interfaz:   

1. Describa en detalle la situación de Chile con respecto a la evolución temporal de las series de tiempo. Destaque patrones y comportamientos que considere relevantes. Compare lo que observa en las distintas series de tiempo. 
1. Considerando la evolución a nivel de semana, conteste ¿En qué fechas ocurre un aumento o disminución notoria de los casos? ¿En qué fechas hay máximos y mínimos?
1. Considerando métricas que sean relativas compare el caso de Chile contra cinco países de la OCDE seleccionados por ustedes. Destaque semejanzas y diferencias considerando las cantidades y los posibles desfases temporales existentes.